In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import SGConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2):
        super(GCN, self).__init__()

        self.conv1 = SGConv(1969, size1, K=2)
        self.fc1 = torch.nn.Linear(size1, size2)
        self.fc2 = torch.nn.Linear(size2, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [ ]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_phyloP/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_phyloP/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_phyloP/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_phyloP/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF", "NOTEARS_thresholdAdaptive"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_phyloP/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_phyloP_SGC/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_phyloP_SGC/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-04 08:52:32,810] A new study created in memory with name: no-name-78e80093-7b84-4053-b49d-f3f026aee732
[I 2023-12-04 08:52:37,346] Trial 0 finished with value: 0.1750551015138626 and parameters: {'size1': 523, 'size2': 395, 'num_epochs': 130, 'lr': 0.007612877874526949}. Best is trial 0 with value: 0.1750551015138626.
[I 2023-12-04 08:52:37,798] Trial 1 finished with value: 0.16169939935207367 and parameters: {'size1': 368, 'size2': 297, 'num_epochs': 132, 'lr': 0.004727346709040634}. Best is trial 1 with value: 0.16169939935207367.
[I 2023-12-04 08:52:37,915] Trial 2 finished with value: 0.15472412109375 and parameters: {'size1': 378, 'size2': 328, 'num_epochs': 31, 'lr': 0.004072089832161823}. Best is trial 2 with value: 0.15472412109375.
[I 2023-12-04 08:52:38,125] Trial 3 finished with value: 0.16172096133232117 and parameters: {'size1': 148, 'size2': 233, 'num_epochs': 66, 'lr': 0.004225031450166573}. Best is trial 2 with value: 0.15472412109375.
[I 2023-12-04 08:52:38,

[I 2023-12-04 08:52:48,662] Trial 4 finished with value: 0.15688836574554443 and parameters: {'size1': 906, 'size2': 303, 'num_epochs': 188, 'lr': 0.0046736374328184525}. Best is trial 4 with value: 0.15688836574554443.
[I 2023-12-04 08:52:48,693] Trial 5 pruned. 
[I 2023-12-04 08:52:48,721] Trial 6 pruned. 
[I 2023-12-04 08:52:48,737] Trial 7 pruned. 
[I 2023-12-04 08:52:48,762] Trial 8 pruned. 
[I 2023-12-04 08:52:48,789] Trial 9 pruned. 
[I 2023-12-04 08:52:48,812] Trial 10 pruned. 
[I 2023-12-04 08:52:48,852] Trial 11 pruned. 
[I 2023-12-04 08:52:48,888] Trial 12 pruned. 
[I 2023-12-04 08:52:48,926] Trial 13 pruned. 
[I 2023-12-04 08:52:49,684] Trial 14 finished with value: 0.16479650139808655 and parameters: {'size1': 478, 'size2': 353, 'num_epochs': 195, 'lr': 0.002966197246654581}. Best is trial 4 with value: 0.15688836574554443.
[I 2023-12-04 08:52:49,722] Trial 15 pruned. 
[I 2023-12-04 08:52:50,426] Trial 16 finished with value: 0.16523832082748413 and parameters: {'size1': 6

[I 2023-12-04 08:53:00,634] Trial 5 pruned. 
[I 2023-12-04 08:53:00,659] Trial 6 pruned. 
[I 2023-12-04 08:53:00,684] Trial 7 pruned. 
[I 2023-12-04 08:53:00,830] Trial 8 pruned. 
[I 2023-12-04 08:53:00,887] Trial 9 finished with value: 0.14527320861816406 and parameters: {'size1': 406, 'size2': 52, 'num_epochs': 13, 'lr': 0.001481319257631624}. Best is trial 9 with value: 0.14527320861816406.
[I 2023-12-04 08:53:00,914] Trial 10 finished with value: 0.6389078497886658 and parameters: {'size1': 422, 'size2': 6, 'num_epochs': 1, 'lr': 0.009360117296120338}. Best is trial 9 with value: 0.14527320861816406.
[I 2023-12-04 08:53:00,941] Trial 11 pruned. 
[I 2023-12-04 08:53:00,964] Trial 12 finished with value: 0.25793465971946716 and parameters: {'size1': 34, 'size2': 336, 'num_epochs': 2, 'lr': 0.004845680159901911}. Best is trial 9 with value: 0.14527320861816406.
[I 2023-12-04 08:53:01,178] Trial 13 finished with value: 0.15345315635204315 and parameters: {'size1': 231, 'size2': 117, 'n

[I 2023-12-04 08:53:12,238] Trial 8 finished with value: 0.16456156969070435 and parameters: {'size1': 157, 'size2': 354, 'num_epochs': 79, 'lr': 0.003719882417731693}. Best is trial 2 with value: 0.1594974249601364.
[I 2023-12-04 08:53:12,495] Trial 9 finished with value: 0.15627332031726837 and parameters: {'size1': 816, 'size2': 295, 'num_epochs': 55, 'lr': 0.0006266889505326633}. Best is trial 9 with value: 0.15627332031726837.
[I 2023-12-04 08:53:12,573] Trial 10 finished with value: 0.1486460268497467 and parameters: {'size1': 1001, 'size2': 284, 'num_epochs': 9, 'lr': 0.0009844689264596551}. Best is trial 10 with value: 0.1486460268497467.
[I 2023-12-04 08:53:12,619] Trial 11 pruned. 
[I 2023-12-04 08:53:12,823] Trial 12 finished with value: 0.15378603339195251 and parameters: {'size1': 853, 'size2': 284, 'num_epochs': 39, 'lr': 0.0017248728624265279}. Best is trial 10 with value: 0.1486460268497467.
[I 2023-12-04 08:53:12,863] Trial 13 pruned. 
[I 2023-12-04 08:53:12,993] Trial

[I 2023-12-04 08:54:29,592] Trial 15 finished with value: 0.19649900496006012 and parameters: {'size1': 446, 'size2': 90, 'num_epochs': 81, 'lr': 0.003363180741033386}. Best is trial 4 with value: 0.1622513383626938.
[I 2023-12-04 08:54:29,645] Trial 16 pruned. 
[I 2023-12-04 08:54:29,717] Trial 17 pruned. 
[I 2023-12-04 08:54:34,034] Trial 18 finished with value: 0.1850091516971588 and parameters: {'size1': 650, 'size2': 306, 'num_epochs': 125, 'lr': 0.001726830209143593}. Best is trial 4 with value: 0.1622513383626938.
[I 2023-12-04 08:54:34,092] Trial 19 pruned. 
[I 2023-12-04 08:54:34,908] A new study created in memory with name: no-name-d71c1693-4c7b-46a2-a80a-acdebc694e51
[I 2023-12-04 08:54:36,873] Trial 0 finished with value: 0.15868069231510162 and parameters: {'size1': 717, 'size2': 321, 'num_epochs': 57, 'lr': 0.00011186768583109907}. Best is trial 0 with value: 0.15868069231510162.
[I 2023-12-04 08:54:40,368] Trial 1 finished with value: 0.18511910736560822 and parameters: 

...

[I 2023-12-04 14:40:23,248] Trial 5 pruned. 
[I 2023-12-04 14:40:23,481] Trial 6 pruned. 
[I 2023-12-04 14:40:23,716] Trial 7 pruned. 
[I 2023-12-04 14:40:44,085] Trial 8 pruned. 
[I 2023-12-04 14:40:44,311] Trial 9 pruned. 
[I 2023-12-04 14:40:44,459] Trial 10 pruned. 
[I 2023-12-04 14:40:44,601] Trial 11 pruned. 
[I 2023-12-04 14:40:47,202] Trial 12 pruned. 
[I 2023-12-04 14:40:47,344] Trial 13 pruned. 
[I 2023-12-04 14:41:03,866] Trial 14 finished with value: 0.21864046156406403 and parameters: {'size1': 440, 'size2': 429, 'num_epochs': 148, 'lr': 0.004265548886075755}. Best is trial 1 with value: 0.1741875559091568.
[I 2023-12-04 14:41:04,014] Trial 15 pruned. 
[I 2023-12-04 14:41:13,438] Trial 16 finished with value: 0.1919342279434204 and parameters: {'size1': 146, 'size2': 312, 'num_epochs': 85, 'lr': 0.0049202740517783495}. Best is trial 1 with value: 0.1741875559091568.
[I 2023-12-04 14:41:13,580] Trial 17 pruned. 
[I 2023-12-04 14:41:13,835] Trial 18 pruned. 
[I 2023-12-04 14

[I 2023-12-04 14:46:57,963] Trial 2 finished with value: 0.17797082662582397 and parameters: {'size1': 854, 'size2': 433, 'num_epochs': 156, 'lr': 0.002584766013823442}. Best is trial 2 with value: 0.17797082662582397.
[I 2023-12-04 14:47:00,183] Trial 3 finished with value: 0.16962994635105133 and parameters: {'size1': 438, 'size2': 362, 'num_epochs': 82, 'lr': 0.0030603796140420245}. Best is trial 3 with value: 0.16962994635105133.
[I 2023-12-04 14:47:02,042] Trial 4 finished with value: 0.19551122188568115 and parameters: {'size1': 798, 'size2': 218, 'num_epochs': 67, 'lr': 0.00460787243444051}. Best is trial 3 with value: 0.16962994635105133.
[I 2023-12-04 14:47:03,523] Trial 5 finished with value: 0.16589747369289398 and parameters: {'size1': 478, 'size2': 434, 'num_epochs': 54, 'lr': 0.0014375502388787452}. Best is trial 5 with value: 0.16589747369289398.
[I 2023-12-04 14:47:03,589] Trial 6 pruned. 
[I 2023-12-04 14:47:05,574] Trial 7 pruned. 
[I 2023-12-04 14:47:05,636] Trial 8 

[I 2023-12-04 14:48:17,427] Trial 2 finished with value: 0.16078060865402222 and parameters: {'size1': 52, 'size2': 119, 'num_epochs': 35, 'lr': 0.0008403440070879871}. Best is trial 2 with value: 0.16078060865402222.
[I 2023-12-04 14:48:17,757] Trial 3 finished with value: 0.3051000237464905 and parameters: {'size1': 393, 'size2': 171, 'num_epochs': 12, 'lr': 0.008964303007794869}. Best is trial 2 with value: 0.16078060865402222.
[I 2023-12-04 14:48:21,797] Trial 4 finished with value: 0.18996591866016388 and parameters: {'size1': 320, 'size2': 317, 'num_epochs': 151, 'lr': 0.002331588975306093}. Best is trial 2 with value: 0.16078060865402222.
[I 2023-12-04 14:48:21,851] Trial 5 pruned. 
[I 2023-12-04 14:48:21,898] Trial 6 pruned. 
[I 2023-12-04 14:48:22,093] Trial 7 finished with value: 0.19821657240390778 and parameters: {'size1': 232, 'size2': 506, 'num_epochs': 7, 'lr': 0.0034852078650889737}. Best is trial 2 with value: 0.16078060865402222.
[I 2023-12-04 14:48:22,165] Trial 8 pr

[I 2023-12-04 14:49:24,012] Trial 5 pruned. 
[I 2023-12-04 14:49:24,884] Trial 6 finished with value: 0.15771141648292542 and parameters: {'size1': 81, 'size2': 161, 'num_epochs': 33, 'lr': 0.002347714636733696}. Best is trial 1 with value: 0.15692564845085144.
[I 2023-12-04 14:49:24,927] Trial 7 pruned. 
[I 2023-12-04 14:49:24,995] Trial 8 pruned. 
[I 2023-12-04 14:49:25,039] Trial 9 pruned. 
[I 2023-12-04 14:49:25,105] Trial 10 pruned. 
[I 2023-12-04 14:49:25,228] Trial 11 finished with value: 0.17014960944652557 and parameters: {'size1': 76, 'size2': 448, 'num_epochs': 4, 'lr': 0.0026195987489962777}. Best is trial 1 with value: 0.15692564845085144.
[I 2023-12-04 14:49:25,278] Trial 12 pruned. 
[I 2023-12-04 14:49:26,047] Trial 13 finished with value: 0.15604296326637268 and parameters: {'size1': 275, 'size2': 507, 'num_epochs': 28, 'lr': 0.002104911242972487}. Best is trial 13 with value: 0.15604296326637268.
[I 2023-12-04 14:49:26,096] Trial 14 pruned. 
[I 2023-12-04 14:49:26,161]

[I 2023-12-04 14:50:42,155] Trial 8 pruned. 
[I 2023-12-04 14:50:42,189] Trial 9 pruned. 
